In [1]:
# medir tiempos
%load_ext autotime

time: 0 ns (started: 2021-06-08 13:59:39 -05:00)


![output](tiempo_secuencial.png)

### Importando la data usando Pyspark 

In [2]:
# Modulo para encontrar pyspark
import findspark
#findspark.init("/usr/local/spark/spark-3.1.1-bin-hadoop2.7")    #para linux
findspark.init()                                                 #para windows

time: 16 ms (started: 2021-06-08 13:59:40 -05:00)


In [3]:
# importamos pyspark
from pyspark import SparkConf, SparkContext
# Variable de configuración
conf = SparkConf().setMaster("local[*]").setAppName("ModeloML")
# iniciamos un contexto spark (solo se ejecuta uno. Para ejecutar otra vez , reiniciar el kernel)
sc = SparkContext(conf = conf)
sc

<SparkContext master=local[*] appName=ModeloML>

time: 4.52 s (started: 2021-06-08 13:59:40 -05:00)


#### DataFrame spark

In [4]:
from pyspark.sql.types import StringType
from pyspark import SQLContext
# le pasamos el contexto anterior
sqlContext = SQLContext(sc)
dfspark = sqlContext.read.format('csv').option("header","true").option("inferSchema","true").load('train.csv')

time: 27.8 s (started: 2021-06-08 13:59:44 -05:00)


In [5]:
# Esquema de los datos
dfspark.printSchema()

root
 |-- key: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)

time: 16 ms (started: 2021-06-08 14:00:12 -05:00)


In [6]:
# Total de cantidad de datos
cant_total=dfspark.count()
cant_total

55423856

time: 6.16 s (started: 2021-06-08 14:00:12 -05:00)


In [7]:
# Tomando una muestra del 4% del total
dfspark_sample = dfspark.sample(fraction = 0.04, withReplacement = False, seed=0)

time: 0 ns (started: 2021-06-08 14:00:18 -05:00)


##### Apartir de aqui trabajamos con una muestra.

In [8]:
# Cantidad de datos para la muestra
cant_muestra=dfspark_sample.count()
cant_muestra

2219408

time: 6.19 s (started: 2021-06-08 14:00:18 -05:00)


### Limpieza de la data

Se procederá a eliminar la columna con la característica "key", debido a que contiene datos innecesarios para lograr el objetivo.

In [9]:
dfspark_sample = dfspark_sample.drop('key')

time: 0 ns (started: 2021-06-08 14:00:25 -05:00)


###### Eliminando Valores Nulos de la tabla

In [10]:
# fare_amount (costo de viaje) no nulos
dfspark_sample = dfspark_sample.filter("fare_amount is not NULL")
# passenger (número de pasajeros) no nulos
dfspark_sample = dfspark_sample.filter("passenger_count is not NULL")
# pickup_datetime (fecha y hora de incio de viaje) no nulos
dfspark_sample = dfspark_sample.filter("pickup_datetime is not NULL")
# pickup (longitud y latitud de inicio de viaje) no nulos
dfspark_sample = dfspark_sample.filter("pickup_longitude is not NULL")
dfspark_sample = dfspark_sample.filter("pickup_latitude is not NULL")
# dropoff (longitud y laitud de fin de viaje) no nulos
dfspark_sample = dfspark_sample.filter("dropoff_longitude is not NULL")
dfspark_sample = dfspark_sample.filter("dropoff_latitude is not NULL")

time: 187 ms (started: 2021-06-08 14:00:25 -05:00)


In [11]:
ncant_muestra=dfspark_sample.count()
ncant_muestra

2219390

time: 23.3 s (started: 2021-06-08 14:00:25 -05:00)


###### Eliminado valores nan y duplicados

In [12]:
# tabla sin valores nan, sin duplicados
dfspark_sample=dfspark_sample.na.drop().dropDuplicates()
# cantidad de  data sin nulos ni nan
cantnn_muestra=dfspark_sample.count()
print("Cantidad de filas eliminadas de la muestra: ",cant_muestra-cantnn_muestra)

Cantidad de filas eliminadas de la muestra:  18
time: 27 s (started: 2021-06-08 14:00:48 -05:00)


In [13]:
# que la data persista en memoria, acelera algunos procesos.
dfspark_sample.persist()

DataFrame[fare_amount: double, pickup_datetime: string, pickup_longitude: double, pickup_latitude: double, dropoff_longitude: double, dropoff_latitude: double, passenger_count: int]

time: 78 ms (started: 2021-06-08 14:01:15 -05:00)


### Estadísticas

In [14]:
import numpy as np
import pandas as pd

time: 328 ms (started: 2021-06-08 14:01:15 -05:00)


In [15]:
# casteamos a pandas
summary=dfspark_sample.describe(["pickup_longitude",
                                 "pickup_latitude",
                                 "dropoff_longitude",
                                 "dropoff_latitude",
                                 "passenger_count",
                                 "fare_amount"]).toPandas()

time: 27.6 s (started: 2021-06-08 14:01:16 -05:00)


In [16]:
summary

,summary,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,fare_amount
0,count,2219390,2219390,2219390,2219390,2219390,2219390
1,mean,-72.50355231460159,39.91779369363991,-72.50697030839864,39.91850726486102,1.6851959322156087,11.352503967306385
2,stddev,13.098889190132796,9.821243657749829,13.244549098861283,10.317917931047642,1.3377753769552685,42.449025079080954
3,min,-3366.527908,-3488.079513,-3366.527908,-3488.079513,0,-52.0
4,max,2497.117435,2964.163855,3211.57975,3333.304575,208,61550.86


time: 15 ms (started: 2021-06-08 14:01:43 -05:00)


In [17]:
# Agrupación por cantidad de pasajeros.
dfspark_sample.groupBy("passenger_count").count().show()

+---------------+-------+
|passenger_count|  count|
+---------------+-------+
|              1|1535882|
|              6|  47132|
|              3|  97242|
|              5| 157254|
|              4|  47158|
|              7|      1|
|              2| 326983|
|              0|   7734|
|            208|      4|
+---------------+-------+

time: 1.95 s (started: 2021-06-08 14:01:43 -05:00)


### Observaciones :
1. Valores de passenger_count imposibles , como 34,49,51,129,208.
2. Precios demasiados elevados debido a la cantidad de pasajeros y negativo(imposible).
3. Cantidad de datos en la que el precio es menor a o igual a 0

In [18]:
# mínimo y máximo para longitud del subconjunto de datos
long_min_i=dfspark_sample.agg({'pickup_longitude': 'min'}).show()
long_max_i=dfspark_sample.agg({'pickup_longitude': 'max'}).show()
long_min_f=dfspark_sample.agg({'dropoff_longitude': 'min'}).show()
long_max_f=dfspark_sample.agg({'dropoff_longitude': 'max'}).show()

# mínimo y máximo para para latitud del subconjuntos de datos
lat_min_i=dfspark_sample.agg({'pickup_latitude': 'min'}).show()
lat_max_i=dfspark_sample.agg({'pickup_latitude': 'max'}).show()
lat_min_f=dfspark_sample.agg({'dropoff_latitude': 'min'}).show()
lat_max_f=dfspark_sample.agg({'dropoff_latitude': 'max'}).show()

+---------------------+
|min(pickup_longitude)|
+---------------------+
|         -3366.527908|
+---------------------+

+---------------------+
|max(pickup_longitude)|
+---------------------+
|          2497.117435|
+---------------------+

+----------------------+
|min(dropoff_longitude)|
+----------------------+
|          -3366.527908|
+----------------------+

+----------------------+
|max(dropoff_longitude)|
+----------------------+
|            3211.57975|
+----------------------+

+--------------------+
|min(pickup_latitude)|
+--------------------+
|        -3488.079513|
+--------------------+

+--------------------+
|max(pickup_latitude)|
+--------------------+
|         2964.163855|
+--------------------+

+---------------------+
|min(dropoff_latitude)|
+---------------------+
|         -3488.079513|
+---------------------+

+---------------------+
|max(dropoff_latitude)|
+---------------------+
|          3333.304575|
+---------------------+

time: 3.26 s (started: 2021-06-0

### Observaciones:
1. Valores para longitud imposibles, ya que longitud varía entre -90 y 90
2. Valores para latitud imposibles, ya que latitud varía entre -180 y 180

# Transformación de la data

In [19]:
# Seleccionar passenger_count de 0-9
dfspark_sample = dfspark_sample.filter("passenger_count < 10")
# Selecionar fare_amount mayor a 0
dfspark_sample = dfspark_sample.filter("fare_amount >= 0")

time: 31 ms (started: 2021-06-08 14:01:49 -05:00)


In [20]:
dfspark_sample.groupBy("passenger_count").count().show()

+---------------+-------+
|passenger_count|  count|
+---------------+-------+
|              1|1535823|
|              6|  47129|
|              3|  97238|
|              5| 157246|
|              4|  47155|
|              7|      1|
|              2| 326972|
|              0|   7734|
+---------------+-------+

time: 2.01 s (started: 2021-06-08 14:01:49 -05:00)


In [21]:
# Filtrando valores grandes para longitud, de tal manera que solo se considerará valores correctos.
dfspark_sample = dfspark_sample.filter("pickup_longitude < 180 and pickup_longitude > -180" )
dfspark_sample = dfspark_sample.filter("dropoff_longitude < 180 and dropoff_longitude > -180")
dfspark_sample = dfspark_sample.filter("pickup_latitude < 90 and pickup_latitude > -90" )
dfspark_sample = dfspark_sample.filter("dropoff_latitude < 90 and dropoff_latitude > -90")

time: 47 ms (started: 2021-06-08 14:01:51 -05:00)


In [22]:
summary_new=dfspark_sample.describe(["pickup_longitude",
                                 "pickup_latitude",
                                 "dropoff_longitude",
                                 "dropoff_latitude"]).toPandas()

time: 1.61 s (started: 2021-06-08 14:01:51 -05:00)


In [23]:
summary_new

,summary,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,count,2219173,2219173,2219173,2219173
1,mean,-72.49600158477304,39.91679455251933,-72.50229931133802,39.91968272022599
2,stddev,10.463045755203852,6.112153680008393,10.439016533276373,6.104146217099324
3,min,-121.91499328613281,-74.017222,-121.9151840209961,-74.177303
4,max,40.840962,74.007413,73.93996,74.95


time: 16 ms (started: 2021-06-08 14:01:52 -05:00)


In [24]:
# Agregamos columnas de diferencias.
from pyspark.sql.functions import abs
dfspark_sample = dfspark_sample.withColumn("dif_latitude",
                                           abs(dfspark_sample['dropoff_latitude']-dfspark_sample['pickup_latitude']))
dfspark_sample = dfspark_sample.withColumn("dif_longitude",
                                           abs(dfspark_sample['dropoff_longitude']-dfspark_sample['pickup_longitude']))

time: 31 ms (started: 2021-06-08 14:01:52 -05:00)


In [25]:
# Creamos la función para hallar la distancia entre dos puntos geográficos
import math
from pyspark.sql.functions import udf, array, col
from pyspark.sql.types import FloatType

def haversine(x):
    lat1=x[0]
    lon1=x[1]
    lat2=x[2]
    lon2=x[3]
    
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

distancia_udf = udf(lambda z: haversine(z), FloatType())
#spark.udf.register('distancia_udf', distancia_udf)
dfspark_sample = dfspark_sample.withColumn('distancia', distancia_udf(array('pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude')))                             

time: 63 ms (started: 2021-06-08 14:01:52 -05:00)


In [26]:
dfspark_sample.select(col('distancia')).show()

+----------+
| distancia|
+----------+
| 1.8683056|
| 1.8191968|
|  1.145462|
| 1.0793539|
| 1.1367035|
|  2.424119|
|  3.829884|
| 1.0900514|
| 5.5276585|
|  3.122764|
| 2.3627946|
| 5.3667502|
| 1.3431213|
|  9.317738|
|  5.013023|
| 4.6022677|
| 1.1546545|
| 20.964258|
| 0.8852747|
|0.17227533|
+----------+
only showing top 20 rows

time: 656 ms (started: 2021-06-08 14:01:52 -05:00)


#### Crear dos columnas día de la semana y hora del viaje.

In [27]:
# funciones que me ayudarán en la transformación.
from datetime import datetime, date, time, timedelta
import calendar
def dia(dia):
    if dia == 1:
        return 'lunes'
    if dia == 2:
        return 'martes'
    if dia == 3:
        return 'miércoles'
    if dia == 4:
        return 'jueves'
    if dia == 5:
        return 'viernes'
    if dia == 6:
        return 'sábado'
    if dia == 7:
        return 'domingo'
    if dia < 1 or dia > 7:
        return 

from pyspark.sql import Row

def dia_semana(row):
    fecha , hora , utc = row.split(" ")
    formato_fecha = "%Y-%m-%d"
    dia_semana = datetime.isoweekday(datetime.strptime(fecha,formato_fecha))
    return dia_semana

def hora(row):
    fecha , hora , utc = row.split(" ")
    formato_hora = "%H:%M:%S"
    hora = datetime.strptime(hora,formato_hora).hour
    return hora
    

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
# convirtiendo las funciones en funciones UDF
udf_dia_semana= udf( lambda z : dia_semana(z))
udf_hora= udf( lambda z : hora(z))

time: 0 ns (started: 2021-06-08 14:01:53 -05:00)


In [28]:
from pyspark.sql.functions import col
dfspark_sample = dfspark_sample.withColumn('dia_semana', 
                                           udf_dia_semana(dfspark_sample['pickup_datetime'] )  )
dfspark_sample = dfspark_sample.withColumn('hora', 
                                           udf_hora(dfspark_sample['pickup_datetime'] )  )

time: 47 ms (started: 2021-06-08 14:01:53 -05:00)


In [29]:
# Castear dia de la semana y hora
dfspark_sample = dfspark_sample.withColumn("dia_semana",
                                           dfspark_sample["dia_semana"].cast("Integer"))
dfspark_sample = dfspark_sample.withColumn("hora",
                                           dfspark_sample["hora"].cast("Integer"))

time: 31 ms (started: 2021-06-08 14:01:53 -05:00)


In [30]:
dfspark_sample.printSchema()

root
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- dif_latitude: double (nullable = true)
 |-- dif_longitude: double (nullable = true)
 |-- distancia: float (nullable = true)
 |-- dia_semana: integer (nullable = true)
 |-- hora: integer (nullable = true)

time: 0 ns (started: 2021-06-08 14:01:53 -05:00)


##### Observación: En este punto la data estaria totalmente ok.

### Estadística de las nuevas variables

In [31]:
clean_summary=dfspark_sample.describe().toPandas()

time: 1min 26s (started: 2021-06-08 14:01:53 -05:00)


In [32]:
columnas = ["summary",
            "pickup_longitude",
            "pickup_latitude",
            "dropoff_longitude",
            "dropoff_latitude",
            "passenger_count",
            "fare_amount",
            "dif_latitude",
            "dif_longitude",
            "dia_semana",
            "hora",
            "distancia"]
clean_summary[columnas]

,summary,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,fare_amount,dif_latitude,dif_longitude,dia_semana,hora,distancia
0,count,2219173,2219173,2219173,2219173,2219173,2219173,2219173,2219173,2219173,2219173,2219173
1,mean,-72.49600158477304,39.91679455251933,-72.50229931133802,39.91968272022599,1.6848177226381178,11.353258637339264,0.09229194952135444,0.16173652033227268,4.042932660049487,13.512919452426647,19.38368660013694
2,stddev,10.463045755203852,6.112153680008393,10.439016533276373,6.104146217099324,1.3087759778931933,42.45082368146176,1.699312422647234,3.1986543317178917,1.9490730146054265,6.517990756130113,365.85874227507634
3,min,-121.91499328613281,-74.017222,-121.9151840209961,-74.177303,0,0.0,0.0,0.0,1,0,0.0
4,max,40.840962,74.007413,73.93996,74.95,7,61550.86,73.961092,89.483332,7,23,9952.922


time: 32 ms (started: 2021-06-08 14:03:20 -05:00)


### Correlación de los atributos

In [ ]:
col_old=["pickup_longitude",
         "pickup_latitude",
         "dropoff_longitude",
         "dropoff_latitude",
         "passenger_count"]
col_new=["dif_latitude",
         "dif_longitude",
         "dia_semana",
         "hora"]
col_pred = ["fare_amount"]

In [ ]:
# Transformación en un vector
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
inputCols = col_old+col_new+col_pred
assembler = VectorAssembler( inputCols=inputCols, outputCol="col_corr")
dfspark_corr = assembler.transform(dfspark_sample)

In [ ]:
dfspark_corr=dfspark_corr.select('col_corr')

In [ ]:
# estudiar la correlación
from pyspark.ml.stat import Correlation

dfspark_corr= Correlation.corr(dfspark_corr, 'col_corr','pearson')

In [ ]:
from pyspark.ml.linalg import DenseMatrix, Vectors
#type(dfspark_cor.collect()[0][0]) # denseMatrix
# Pasamos la matrix como un array.
array_corr=dfspark_corr.collect()[0][0].toArray()

In [ ]:
pdf_corr= pd.DataFrame(array_corr, columns=inputCols, index=inputCols)
mask = ~(pdf_corr>-0.3) | ~(pdf_corr<0.3)
round(pdf_corr,10).style.background_gradient()

### Observación:
###### La variable a predecir fare_amount itnee muy baja correlación con las demás variables. Su mayor correlación es con passenger_count y hora. Pero si observamos correlación entre las demás variables. Definitivamente no podemos utilizar un modelo lineal.

### Visualización de la data.

In [ ]:
# Pasamos toda la data a Pandas
pandasData = dfspark_sample.toPandas()

In [ ]:
#Manera alternativa de pasar la data a Pandas
# import pandas as pd
# from pyspark.sql import DataFrame

# # Wrapper for seamless Spark's serialisation
# def spark_to_pandas(spark_df: DataFrame) -> pd.DataFrame:
#     """
#     PySpark toPandas realisation using mapPartitions
#     much faster than vanilla version
#     fork: https://gist.github.com/lucidyan/1e5d9e490a101cdc1c2ed901568e082b
#     origin: https://gist.github.com/joshlk/871d58e01417478176e7
#     :param spark_df:
#     :return:
#     """
    
#     def _map_to_pandas(rdds) -> list:
#         """ Needs to be here due to pickling issues """
#         return [pd.DataFrame(list(rdds))]

#     def _to_pandas(df: DataFrame, n_partitions: int = None) -> pd.DataFrame:
#         """
#         Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
#         repartitioned if `n_partitions` is passed.
#         :param df:
#         :param n_partitions:
#         :return:
#         """
#         if n_partitions is not None:
#             df = df.repartition(n_partitions)
#         df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()  # type: pd.DataFrame
#         df_pand = pd.concat(df_pand)
#         df_pand.columns = df.columns
#         return df_pand

#     return _to_pandas(spark_df)

# pandasData = spark_to_pandas(dfspark_sample)

In [ ]:
#Mostrando la data
display(pandasData)

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set(style="ticks" , color_codes = True)
var = ["dia_semana"]
# 1 inches = 96px
g = sns.pairplot(pandasData[:10], vars=var, diag_kind="hist", hue='dia_semana', height=4, aspect=4)

In [ ]:
#Cuanto de la variable objetivo va variando segun los dias de la semana
import matplotlib.pyplot as plt

carac1=pandasData['dia_semana']
objet=pandasData['fare_amount']
plt.plot(carac1, objet, 'o')
plt.xlabel("Característica dia de la semana")
plt.ylabel("Objetivo")

In [ ]:
#como los pasajeros se distribuyen a traves de las horas
pandasData.groupby('hora')['passenger_count'].sum().plot(kind='barh',legend='Reverse',figsize=(10,10))
plt.xlabel('')

In [ ]:
#como se reparten los pasajeros en funcion de la hora
pandasData.passenger_count.groupby(pandasData.hora).sum().plot(kind='pie',cmap='Paired',figsize=(12,8))
plt.axis('equal')

In [ ]:
#Dispersion de las características
caracteristicas=pandasData[['fare_amount','pickup_longitude','pickup_latitude','dropoff_longitude']]
sns.set(style="ticks", color_codes=True)
g= sns.pairplot(caracteristicas,hue='fare_amount',palette='Spectral')

In [ ]:
# cerramos la sesión spark
sc.stop()